# Common routine for CIFAR10 (preparation)

In [39]:
import torch, torchvision; from transformers import Trainer, TrainingArguments, EarlyStoppingCallback; device = 'cuda' if torch.cuda.is_available() else 'cpu'
B = 1024//4
tf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset= torchvision.datasets.CIFAR10(root=".", train=True, download=True, transform=tf); trainloader= torch.utils.data.DataLoader(trainset,batch_size=B, shuffle=True, num_workers=4)
testset = torchvision.datasets.CIFAR10(root=".", train=False,download=True, transform=tf); testloader = torch.utils.data.DataLoader(testset, batch_size=B, shuffle=False,num_workers=4)

class Net(torch.nn.Module): # modified from https://www.kaggle.com/code/faressayah/cifar-10-images-classification-using-cnns-88
    def __init__(self):
        super().__init__()
        self.pool = torch.nn.MaxPool2d(2, 2); p=0.0; # p=1/6
        self.dropout1 = torch.nn.Dropout(p)
        self.dropout2 = torch.nn.Dropout(p)
        self.dropout3 = torch.nn.Dropout(p)
        self.dropout4 = torch.nn.Dropout(p)
        self.conv1 = torch.nn.Conv2d( 3, 32, 3, 1, 1); self.batch_norm1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 32, 3, 1, 1); self.batch_norm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 64, 3, 1, 1); self.batch_norm3 = torch.nn.BatchNorm2d(64)
        self.conv4 = torch.nn.Conv2d(64, 64, 3, 1, 1); self.batch_norm4 = torch.nn.BatchNorm2d(64)
        self.conv5 = torch.nn.Conv2d(64, 128,3, 1, 1); self.batch_norm5 = torch.nn.BatchNorm2d(128)
        self.conv6 = torch.nn.Conv2d(128,128,3, 1, 1); self.batch_norm6 = torch.nn.BatchNorm2d(128)
        self.fc1 = torch.nn.Linear(128 * 4 * 4, 64)
        self.fc2 = torch.nn.Linear(64, 10)
        self.criterion = torch.nn.CrossEntropyLoss()
    def forward(self, input, target):
        x = self.batch_norm1(torch.nn.functional.relu(self.conv1(input)))
        x = self.pool(self.batch_norm2(torch.nn.functional.relu(self.conv2(x))))
        x = self.dropout1(x)
        x = self.batch_norm3(torch.nn.functional.relu(self.conv3(x)))
        x = self.pool(self.batch_norm4(torch.nn.functional.relu(self.conv4(x))))
        x = self.dropout2(x)
        x = self.batch_norm5(torch.nn.functional.relu(self.conv5(x)))
        x = self.pool(self.batch_norm6(torch.nn.functional.relu(self.conv6(x))))
        x = self.dropout3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)
        return torch.nn.functional.cross_entropy(x, target), x

def compute_metrics(logits):
    _, predicted = torch.max(torch.tensor(logits[0]), 1)
    return {'accuracy': (predicted == torch.tensor(testset.targets) ).sum().item() / len(testset.targets)}

def collator(X):
    temp = torch.Tensor([X[i][1] for i in range(len(X))]).to(dtype=torch.int64)
    temp1 = torch.stack([X[i][0] for i in range(len(X))])
    return {'input': temp1, 'target': temp}

Files already downloaded and verified
Files already downloaded and verified


# Normal Model

In [3]:
# Normal Models (hyperparameteters tuned)

# Best p for DropOut searched, (ep10 fixed)
# p=0: 85.39%  85.15% again
# p=0.1: 85.81%
# p=0.14: 86.56% best!
# p=0.15: 86.14% 
# p=1/6: 86.09%
# p=0.18: 86.54%
# p=0.2: 86.25%
# p=0.225: 85.81%
# p=0.25: 85.79%
# p=0.4: 84.43%

## Best max number of epochs searched
# 74.2%  5 ep # Max_N_ep 512
# 72.4%  3 ep # Max_N_ep 256
# 70.5%  4 ep # Max_N_ep 128
# 74.0%  6 ep # Max_N_ep 64
# 79.9% 13 ep # Max_N_ep 32
# 80.5%  7 ep # Max_N_ep 16
# 83.9% 11 ep # Max_N_ep 12
# 85.4% 10 ep # Max_N_ep 10    ->   BEST
# 85.1%  8 ep # Max_N_ep 8
# 84.5%  6 ep # Max_N_ep 6
# 82.8%  4 ep # Max_N_ep 4
# 74.1%  2 ep # Max_N_ep 2
# 61.2%  1 ep # Max_N_ep 1

## Best weight decay searched
# 18.0% 90.19/4 sec # WD 16
# 78.7% 216.0/10sec # WD 4 unstable x
# 65.4% 87.13/4 sec # WD 1
# 85.7% 208.3/10sec # WD 4/5 fp16
# 85.8% 217.6/10sec # WD 3/4 slightly unstable x
# 85.6% 209.3/10sec # WD 2/3 fp16
# 85.4% 220.6/10sec # WD 1/2  ->   FIX to 1/2
# 82.9% 177.6/8 sec # WD 1/3
# 84.2% 195.5/9 sec # WD 1/4
# 83.4% 173.9/8 sec # WD 1/6
# 84.2% 194.5/9 sec # WD 1/4/2
# 81.9% 154.2/7 sec # WD 1/4/4
# 81.3% 153.6/7 sec # WD 1/4/4/4
# 82.3% 176.5/8 sec # WD 1/4/4/4/4/4
# 82.2% 154.0/7 sec # WD 0   weight_decay=0
# 82.0% 150.3/7 sec # no WD specified

## Best learning rate searched
# 79.6% 175.8/8 sec # lr20e-3
# 81.4% 176.4/8 sec # lr14e-3
# 81.5% 177.8/8 sec # lr9e-3
# 82.2% 154.3/7 sec # lr7e-3 again   ->   FIX to 7e-3
# 82.3% 177.8/8 sec # lr5e-3
# 81.7% 150.8/7 sec # lr3e-3
# 80.7% 106.2/5 sec # lr1e-3
# 79.2% 150.7/7 sec # lr0.5e-3
# 75.2% 152.4/7 sec # lr0.25e-3
# 71.5% 220.6/10 sec # lr0.1e-3

## Best number of filters searched
# 79.8% 165.1/8 sec 0.4G # 16-32-64 x

## Best number of neurons in hidden layer searched  ## default: Ep10, WD1/4/4/4/4, lr7e-3, 32-64-128
#x10.0% for 4 neurons 130.1/6 sec 0.7G
#x80.4% for 8 neurons 174.2/8 sec 0.7G or 10.0%
# 81.8% for 16 neurons 176.7/8 sec 0.7G
# 81.6% for 32 neurons 153.7/7 sec
# 82.1% for 64 neurons 175.8/8 sec   ->   FIX to 64 neurons
# 80.9% for 128 neurons ?sec
# 81.8% for 256 neurons ?sec
# 82.3% for 512 neurons ?sec
# 81.8% for 1024 neurons 172/8 sec
# 82.3% for 2048 neurons 183/9 sec
# 80.9% for 4096 neurons 183.8/9 sec

In [34]:
# p=1/6
model = Net().to(device)
training_args = TrainingArguments(num_train_epochs=10, weight_decay=1/2, learning_rate=7e-3, lr_scheduler_type="linear", fp16=True, load_best_model_at_end=True, output_dir='./test', logging_strategy="epoch", save_strategy='epoch', per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='epoch', report_to='none')
trainer = Trainer(model=model, compute_metrics=compute_metrics, data_collator=collator, args=training_args, train_dataset=trainset, eval_dataset=testset, callbacks = [EarlyStoppingCallback(2,0.0)]); trainer.can_return_loss = True;
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.563900,1.273380,0.534800
2,1.080200,0.901626,0.686800
3,0.849700,0.866347,0.703000
4,0.728800,0.796626,0.720700
5,0.635300,0.647749,0.782000
6,0.567400,0.617481,0.789600
7,0.496600,0.551259,0.812100
8,0.420100,0.486332,0.835600
9,0.336100,0.453437,0.845400
10,0.249400,0.419907,0.860900


TrainOutput(global_step=1960, training_loss=0.6927466820697395, metrics={'train_runtime': 196.6962, 'train_samples_per_second': 2541.991, 'train_steps_per_second': 9.965, 'total_flos': 0.0, 'train_loss': 0.6927466820697395, 'epoch': 10.0})

In [40]:
# p=0
model = Net().to(device)
training_args = TrainingArguments(num_train_epochs=10, weight_decay=1/2, learning_rate=7e-3, lr_scheduler_type="linear", fp16=True, load_best_model_at_end=True, output_dir='./test', logging_strategy="epoch", save_strategy='epoch', per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='epoch', report_to='none')
trainer = Trainer(model=model, compute_metrics=compute_metrics, data_collator=collator, args=training_args, train_dataset=trainset, eval_dataset=testset, callbacks = [EarlyStoppingCallback(2,0.0)]); trainer.can_return_loss = True;
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.521600,1.167548,0.570300
2,0.905100,0.905943,0.686300
3,0.696800,0.866236,0.696900
4,0.585800,0.778480,0.734800
5,0.493000,0.725188,0.754900
6,0.412800,0.607915,0.797600
7,0.329100,0.541160,0.817800
8,0.228400,0.500386,0.837800
9,0.127700,0.507056,0.844000
10,0.039100,0.499806,0.851500


TrainOutput(global_step=1960, training_loss=0.5339338682135757, metrics={'train_runtime': 205.7295, 'train_samples_per_second': 2430.376, 'train_steps_per_second': 9.527, 'total_flos': 0.0, 'train_loss': 0.5339338682135757, 'epoch': 10.0})

# Forgetter (Without last drop now, Try last drop in the future work)

In [ ]:
## Forgetter with l=7ep fixed
# p=0:   87.05% @10th
# p=0.1: 87.51% @12th(max)
# p=0.14: 87.51% @7th
# p=0.15: 87.34%@6th
# p=1/6:    again   87.81@11th            87.96%@12th(max) without data augmentation
# p=0.18: 87.70% @12th(max)
# p=0.2: 87.68@12th(max)
# p=0.225: 87.79@8th
# p=0.25: 87.30% @7th
# p=0.4: 85.58% @10th

## p=0
# l= 1ep 84.49% @21th/36
# l= 3ep 86.40% @9th
# l= 5ep 86.71% @11th
# l= 7ep: 87.05% @10th
# l= 9ep: 86.92% @6th
# l=11ep: 86.88% @4th
# l=13ep: 86.88% @8th
# l=15ep: 86.62% @2nd
# l=17ep: 85.90% @4th
# l=19ep: 86.96% @5th

In [18]:
# p=1/6: 87.96%@12th without data augmentation
model = Net().to(device); torch.cuda.empty_cache()
for i in range(4 *3):
    training_args = TrainingArguments(num_train_epochs=7, weight_decay=1/2, learning_rate=7e-3, lr_scheduler_type="linear", fp16=True, load_best_model_at_end=True, output_dir='./test', logging_strategy="epoch", save_strategy='epoch', per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='epoch', report_to='none')
    trainer = Trainer(model=model, compute_metrics=compute_metrics, data_collator=collator, args=training_args, train_dataset=trainset, eval_dataset=testset); trainer.can_return_loss = True;
    trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.552300,1.305877,0.535900
2,1.024100,0.926753,0.678400
3,0.806200,0.720201,0.750100
4,0.678300,0.650388,0.777500
5,0.569500,0.556564,0.811600
6,0.456500,0.509839,0.829900
7,0.348400,0.444927,0.853000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.735000,0.761184,0.739100
2,0.656100,0.767646,0.746000
3,0.596200,0.616969,0.786300
4,0.533600,0.648708,0.781800
5,0.455400,0.543349,0.816600
6,0.368800,0.477860,0.843300
7,0.264200,0.412849,0.863200


Epoch,Training Loss,Validation Loss,Accuracy
1,0.730600,0.763553,0.740900
2,0.642100,0.769258,0.735500
3,0.578400,0.661447,0.779500
4,0.508600,0.623779,0.787800
5,0.437700,0.556988,0.814700
6,0.351700,0.453311,0.849700
7,0.246000,0.402951,0.866000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.697600,0.946135,0.688400
2,0.620200,0.753916,0.740700
3,0.553000,0.653013,0.782000
4,0.494400,0.645014,0.784900
5,0.424900,0.511852,0.828700
6,0.340000,0.418717,0.861500
7,0.237800,0.392263,0.871100


Epoch,Training Loss,Validation Loss,Accuracy
1,0.693500,0.850486,0.711600
2,0.609200,0.669669,0.767400
3,0.553800,0.729484,0.764700
4,0.498700,0.598517,0.794300
5,0.420400,0.488080,0.833800
6,0.334400,0.443566,0.852400
7,0.236400,0.387291,0.874000


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.671300,0.827838,0.725200
2,0.608400,0.848556,0.711100
3,0.547000,0.711860,0.758400
4,0.484000,0.602690,0.801500
5,0.406700,0.522167,0.828400
6,0.321100,0.414746,0.861800
7,0.222800,0.379910,0.877700


Epoch,Training Loss,Validation Loss,Accuracy
1,0.664800,0.890706,0.690900
2,0.597000,0.925404,0.700700
3,0.543200,0.621693,0.787800
4,0.486000,0.661027,0.779600
5,0.406900,0.480840,0.836200
6,0.320400,0.438217,0.854200
7,0.222800,0.387630,0.873300


Epoch,Training Loss,Validation Loss,Accuracy
1,0.680700,0.922429,0.686100
2,0.601500,0.776766,0.737200
3,0.540600,0.839467,0.729500
4,0.482200,0.691438,0.773300
5,0.402900,0.558677,0.817200
6,0.325000,0.413350,0.861100
7,0.221400,0.383404,0.873400


Epoch,Training Loss,Validation Loss,Accuracy
1,0.679800,0.738306,0.748400
2,0.604400,0.959916,0.692200
3,0.529000,0.665175,0.780300
4,0.481000,0.575141,0.803700
5,0.408900,0.531933,0.818700
6,0.319600,0.479334,0.841400
7,0.219900,0.381543,0.875200


Epoch,Training Loss,Validation Loss,Accuracy
1,0.689200,0.929882,0.705600
2,0.601500,0.814252,0.726700
3,0.538300,0.696872,0.759600
4,0.475800,0.564464,0.810700
5,0.404600,0.532331,0.828900
6,0.316600,0.412615,0.859400
7,0.217100,0.375071,0.879600


In [38]:
# p=0: 86.66%
model = Net().to(device); torch.cuda.empty_cache()
for i in range(4 *3):
    training_args = TrainingArguments(num_train_epochs=7, weight_decay=1/2, learning_rate=7e-3, lr_scheduler_type="linear", fp16=True, load_best_model_at_end=True, output_dir='./test', logging_strategy="epoch", save_strategy='epoch', per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='epoch', report_to='none')
    trainer = Trainer(model=model, compute_metrics=compute_metrics, data_collator=collator, args=training_args, train_dataset=trainset, eval_dataset=testset); trainer.can_return_loss = True;
    trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.473100,1.237151,0.566800
2,0.915200,1.007223,0.652300
3,0.693000,0.759306,0.732200
4,0.557200,0.716968,0.755600
5,0.441500,0.577269,0.802500
6,0.311100,0.506118,0.828600
7,0.161700,0.492942,0.840300


Epoch,Training Loss,Validation Loss,Accuracy
1,0.623700,0.809500,0.721200
2,0.554100,0.754241,0.744500
3,0.484200,0.683308,0.766500
4,0.412300,0.661162,0.771900
5,0.320300,0.587605,0.807400
6,0.204400,0.492132,0.844100
7,0.078000,0.473661,0.854900


Epoch,Training Loss,Validation Loss,Accuracy
1,0.613600,0.999666,0.684100
2,0.535200,0.716151,0.763300
3,0.445100,0.678258,0.776000
4,0.378100,0.664880,0.784600
5,0.291400,0.538364,0.822000
6,0.180800,0.482828,0.841300
7,0.065500,0.467647,0.856200


Epoch,Training Loss,Validation Loss,Accuracy
1,0.604800,0.749729,0.741400
2,0.509400,0.674041,0.773200
3,0.439600,0.771994,0.737500
4,0.365000,0.656201,0.787800
5,0.271800,0.517912,0.825400
6,0.163900,0.485493,0.847700
7,0.058100,0.463024,0.863500


Epoch,Training Loss,Validation Loss,Accuracy
1,0.572000,0.689065,0.767800
2,0.499300,0.658060,0.776000
3,0.425300,0.641545,0.784800
4,0.353100,0.587337,0.807800
5,0.267100,0.568970,0.814900
6,0.159500,0.481320,0.847800
7,0.055500,0.451046,0.864800


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.553800,0.679087,0.764200
2,0.479600,0.623457,0.784900
3,0.411100,0.682461,0.770800
4,0.336800,0.597549,0.803800
5,0.246600,0.517347,0.833200
6,0.148100,0.484265,0.847600
7,0.050300,0.468071,0.861200


Epoch,Training Loss,Validation Loss,Accuracy
1,0.564700,0.921380,0.689000
2,0.484500,0.769248,0.742100
3,0.417500,0.692958,0.768600
4,0.337900,0.554973,0.813300
5,0.251600,0.550261,0.817400
6,0.146300,0.487148,0.850900
7,0.047200,0.457577,0.866600


Epoch,Training Loss,Validation Loss,Accuracy
1,0.553200,0.935497,0.685200
2,0.484600,0.809507,0.737600
3,0.415000,0.772614,0.744400
4,0.335700,0.600651,0.799700
5,0.249600,0.550885,0.822900
6,0.141900,0.493474,0.847000
7,0.048000,0.460799,0.865600


Epoch,Training Loss,Validation Loss,Accuracy
1,0.567000,0.788326,0.727100
2,0.478900,0.796516,0.725800
3,0.406700,0.772040,0.732700
4,0.332400,0.557793,0.811000
5,0.248000,0.565350,0.818600
6,0.141900,0.517967,0.844300
7,0.049500,0.458845,0.865900


Epoch,Training Loss,Validation Loss,Accuracy
1,0.581700,0.773610,0.733200
2,0.489600,0.661982,0.779400
3,0.411800,0.748099,0.748300
4,0.337000,0.640041,0.783900
5,0.252400,0.643277,0.804000
6,0.147700,0.486543,0.851200
7,0.050300,0.465223,0.862400
